In [1]:
import praw 
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import numpy as np
import yfinance as yf
import time
from datetime import datetime, timedelta
import requests

import utils

from nltk.tokenize import word_tokenize

In [2]:
#Basic information about stocks
stocks = pd.read_csv("/Users/arnavprasad/Google Drive/Documents/Coding/Reddit stocks/NASDAQ_names.csv", index_col="Symbol")[["Name"]]

subreddit = "thewallstreet"


save_loc = "/Users/arnavprasad/Documents/GitHub/reddit-stocks/Figures"

In [61]:
#Set date range

start = "2020-05-01"
end = "2020-07-01"

date_range = pd.date_range(start, end)

In [67]:
tickers = {}

for date in date_range:
    url = utils.create_url(subreddit, date)

    data = requests.get(url).json()["data"]

    for comment in data:

        body = comment["body"]
        body_words = word_tokenize(body)

        for symbol in stocks.index:

            if symbol in body_words:

                upvotes = comment["score"]
                polarity = utils.getPolarity(body)

                weighted_sentiment = upvotes * polarity

                if symbol in tickers.keys():

                    if date in tickers[symbol].keys():

                        tickers[symbol][date]["comments"].append(body)
                        tickers[symbol][date]["p_list"].append(weighted_sentiment)

                    else:

                        tickers[symbol][date] = {"score": None, "change": None, "comments": [body], "p_list": [weighted_sentiment]}


                else:

                    tickers[symbol] = {date: {"score": None, "change": None, "comments": [body], "p_list": [weighted_sentiment]}}


In [68]:
for symbol in tickers.keys():

    for date in tickers[symbol].keys():

        p_list = tickers[symbol][date]["p_list"]
        score = utils.sentiment_score(np.mean(p_list))
        tickers[symbol][date]["score"] = score

        try:

            entry = yf.Ticker(symbol).history(start=date, end=date + timedelta(days=1)).iloc[0]

            change = entry["Close"] - entry["Open"]

            tickers[symbol][date]["change"] = change

        except IndexError:

            tickers[symbol][date]["change"] = "market closed"

    for date in tickers[symbol].copy().keys():

        if tickers[symbol][date]["change"] == "market closed":

            del tickers[symbol][date]["p_list"]

            del tickers[symbol][date]

        else:

            del tickers[symbol][date]["p_list"]




- AMZN: No data found for this date range, symbol may be delisted
- AMZN: No data found for this date range, symbol may be delisted
- F: No data found for this date range, symbol may be delisted
- F: No data found for this date range, symbol may be delisted
- T: No data found for this date range, symbol may be delisted
- T: No data found for this date range, symbol may be delisted
- W: No data found for this date range, symbol may be delisted
- W: No data found for this date range, symbol may be delisted
- PINS: No data found for this date range, symbol may be delisted
- CEO: No data found for this date range, symbol may be delisted
- ATH: No data found for this date range, symbol may be delisted
- ATH: No data found for this date range, symbol may be delisted
- TSLA: No data found for this date range, symbol may be delisted
- TSLA: No data found for this date range, symbol may be delisted
- TSLA: No data found for this date range, symbol may be delisted
- CD: Data doesn't exist for st

In [77]:
#Example
list(tickers.items())[:2]

[('JPM',
  {Timestamp('2020-05-01 00:00:00', freq='D'): {'score': -0.03666184419716878,
    'change': 0.15700267281336266,
    'comments': ["Wells Fargo to stop offering new HELOCs. JPM adopted the same policy a while back. WFC is the largest home lender in the U.S by $ volume so obviously they don't want to take on more risk but it's not bullish at all. They know consumer finances will be disgusting going forward and RE will keep going down. Look for other banks to do the same and if shit keeps deteriorating they could easily start pulling existing lines like in 2008."]},
   Timestamp('2020-05-05 00:00:00', freq='D'): {'score': 0.12747470713057618,
    'change': 0.1177520021375642,
    'comments': ['New to Market profile? [Check out this guide](http://www.ranchodinero.com/volume-tpo-essentials/) and [this book](https://web.archive.org/web/20170829185705/https://www.cmegroup.com/education/interactive/marketprofile/handbook.pdf)\n\n \n\nAdditional tickers (PM or reply in these threads i